<a href="https://colab.research.google.com/github/alviansyahburhani/Deteksi-Kendaraan-CNN/blob/main/testMN_BagusBBOXnya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Bagian Baru

In [2]:
# !unrar x "/content/drive/MyDrive/cnn od/sign language.rar" "/content/drive/MyDrive/cnn od/"

In [3]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from collections import Counter


In [4]:
# Path to your dataset directory
dataset_dir = '/content/drive/MyDrive/DataSet-Kendaraan/Training'

In [5]:
# Fungsi untuk memuat gambar dan anotasi
def load_dataset(dataset_dir):
    images = []
    annotations = []
    classes_set = set()

    # Memuat gambar dan anotasi XML yang sesuai
    image_dir = os.path.join(dataset_dir, 'images')
    label_dir = os.path.join(dataset_dir, 'labels')

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(image_dir, filename)
            annotation_path = os.path.join(label_dir, os.path.splitext(filename)[0] + '.xml')

            # Membaca gambar
            image = cv2.imread(img_path)
            height, width, _ = image.shape

            # parsing anotasi XML
            tree = ET.parse(annotation_path)
            root = tree.getroot()

            # Mengambil ukuran gambar dari XML
            image_size = root.find('size')
            img_width = int(image_size.find('width').text)
            img_height = int(image_size.find('height').text)

            # Mengubah ukuran gambar jika diperlukan (opsional)
            #image = cv2.resize(image, (320, 320))

            # Mengambil koordinat bounding box dan label
            for obj in root.findall('object'):
                obj_name = obj.find('name').text
                classes_set.add(obj_name)  # Menambahkan kelas ke set kelas
                bbox = obj.find('bndbox')
                xmin = int(bbox.find('xmin').text) / img_width * 320
                xmax = int(bbox.find('xmax').text) / img_width * 320
                ymin = int(bbox.find('ymin').text) / img_height * 320
                ymax = int(bbox.find('ymax').text) / img_height * 320

                annotations.append({
                    'image_path': img_path,
                    'label': obj_name,
                    'bbox': [xmin, ymin, xmax, ymax],
                })

            images.append(image)

    classes_list = sorted(list(classes_set))  # Mengubah set menjadi daftar kelas yang terurut
    num_classes = len(classes_list)

    return images, annotations, num_classes, classes_list

In [6]:
# Fungsi untuk menampilkan gambar dengan bounding box
def display_images_with_bboxes(images, annotations, classes_list, num_samples=5):
    plt.figure(figsize=(15, 15))
    for i in range(num_samples):
        image = images[i]
        ann = annotations[i]
        bbox = ann['bbox']
        label = ann['label']
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                                          edgecolor='red', facecolor='none', linewidth=2))
        plt.title(classes_list[label])
    plt.show()

In [8]:
# Memuat dataset
images, annotations, num_classes, classes_list = load_dataset(dataset_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DataSet-Kendaraan/Training/images'

In [ ]:
# Mengubah label menjadi nilai numerik
class_to_index = {cls: idx for idx, cls in enumerate(classes_list)}
for annotation in annotations:
    annotation['label'] = class_to_index[annotation['label']]

In [ ]:
# Menghitung jumlah setiap kelas dalam dataset
label_counts = Counter([ann['label'] for ann in annotations])

# Menampilkan statistik jumlah dataset berdasarkan kelas
print("\nStatistik jumlah dataset berdasarkan kelas:")
for cls, count in label_counts.items():
    print(f"{classes_list[cls]}: {count} gambar")

In [ ]:
# Mempersiapkan data untuk pelatihan
X = np.array(images)
y_bbox = np.array([ann['bbox'] for ann in annotations])
y_class = np.array([ann['label'] for ann in annotations])

In [ ]:
# Membagi data menjadi set pelatihan dan validasi
X_train, X_val, y_bbox_train, y_bbox_val, y_class_train, y_class_val, annotations_train, annotations_val = train_test_split(
    X, y_bbox, y_class, annotations, test_size=0.2, random_state=42)

In [ ]:
# Menampilkan beberapa gambar dengan bounding box
display_images_with_bboxes(X_train, annotations_train, classes_list, num_samples=5)

In [ ]:
# Menampilkan informasi dataset
print(f"Jumlah gambar: {len(X)}")
print(f"Jumlah kelas: {num_classes}")
print(f"Dimensi gambar: {X.shape[1:]}")
print(f"Label kelas: {classes_list}")


In [ ]:
# Mendefinisikan model MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(320, 320, 3))


In [ ]:
# Menambahkan layer custom untuk regresi bounding box dan prediksi kelas
x = base_model.output
x = GlobalAveragePooling2D()(x)


In [ ]:
#from tensorflow.keras.layers import Dropout

#Menambahkan Dropout ke layer yang sesuai
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dropout(0.2)(x)  # Contoh: dropout 50%

In [ ]:
# Output untuk regresi bounding box
bbox_output = Dense(4, activation='linear', name='bbox_output')(x)


In [ ]:
# Output untuk prediksi kelas
class_output = Dense(num_classes, activation='softmax', name='class_output')(x)


In [ ]:
# Menggabungkan output
model = Model(inputs=base_model.input, outputs=[bbox_output, class_output])

In [ ]:
# Menampilkan bentuk model
model.summary()


In [ ]:
# Mengompilasi model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'bbox_output': 'mean_squared_error', 'class_output': 'sparse_categorical_crossentropy'},
              metrics={'bbox_output': 'mse', 'class_output': 'accuracy'})

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping

# Definisikan Early Stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

from tensorflow.keras.callbacks import ModelCheckpoint

# Path untuk menyimpan model
checkpoint_path = '/content/drive/MyDrive/Dataset/arya6/model_checkpoint2.h5'

# Callback untuk menyimpan model ketika val_accuracy meningkat
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_class_output_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max',
    save_weights_only=False,
    save_freq='epoch'
)

# Melatih model dengan ModelCheckpoint
history = model.fit(
    X_train, {'bbox_output': y_bbox_train, 'class_output': y_class_train},
    validation_data=(X_val, {'bbox_output': y_bbox_val, 'class_output': y_class_val}),
    epochs=150,
    batch_size=32,
    callbacks=[checkpoint]  # Menambahkan callback ModelCheckpoint
)


In [ ]:
# Save the trained model
#model.save('/content/drive/MyDrive/Materi/jari/CNN_Arya2.h5')

In [ ]:
import json

# Simpan classes_list ke dalam file JSON di Google Drive
def save_classes_list(classes_list, save_path):
    with open(save_path, 'w') as f:
        json.dump(classes_list, f)
    print(f"File JSON berhasil disimpan di: {save_path}")

# Contoh penggunaan:
# Ganti path sesuai dengan direktori yang Anda inginkan di Google Drive
save_path = '/content/drive/MyDrive/Dataset/arya6/classes_list.json'
save_classes_list(classes_list, save_path)

In [ ]:
import pickle

# Daftar kelas yang digunakan selama pelatihan
classes_list = [...]  # Daftar kelas yang sama dengan yang digunakan saat pelatihan

# Simpan daftar kelas ke file PKL
with open('/content/drive/MyDrive/Dataset/arya6/classes_list.pkl', 'wb') as f:
    pickle.dump(classes_list, f)

In [ ]:
classes_list

In [ ]:
# Muat daftar kelas dari file JSON
with open('/content/drive/MyDrive/Dataset/arya6/classes_list.json', 'r') as f:
    classes_list = json.load(f)

    print(classes_list)

In [ ]:
# Menampilkan history pelatihan
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['bbox_output_loss'], label='Bounding Box Loss')
plt.plot(history.history['val_bbox_output_loss'], label='Val Bounding Box Loss')
plt.legend()
plt.title('Bounding Box Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['class_output_accuracy'], label='Class Accuracy')
plt.plot(history.history['val_class_output_accuracy'], label='Val Class Accuracy')
plt.legend()
plt.title('Class Accuracy')
plt.show()

In [ ]:
# Fungsi untuk menampilkan gambar dengan bounding box dan nilai prediksi
def display_validation_results(X_val, y_bbox_val, y_class_val, classes_list, model, num_samples=10, samples_per_row=5):
    predictions = model.predict(X_val)
    bbox_predictions, class_predictions = predictions[0], predictions[1]

    num_rows = (num_samples + samples_per_row - 1) // samples_per_row  # Menghitung jumlah baris yang diperlukan
    plt.figure(figsize=(15, 3 * num_rows))  # Menyesuaikan ukuran gambar berdasarkan jumlah baris

    for i in range(num_samples):
        image = X_val[i]
        bbox_pred = bbox_predictions[i]
        class_pred = np.argmax(class_predictions[i])

        plt.subplot(num_rows, samples_per_row, i + 1)
        plt.imshow(cv2.cvtColor(image.astype('uint8'), cv2.COLOR_BGR2RGB))

        # Menggambar bounding box berdasarkan prediksi
        xmin, ymin, xmax, ymax = bbox_pred
        plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                          edgecolor='green', facecolor='none', linewidth=2))
        plt.title(f'{classes_list[class_pred]}\nActual: {classes_list[y_class_val[i]]}')

        # Menambahkan nilai akurasi di subplot
        plt.xlabel(f'Class Accuracy: {class_predictions[i][y_class_val[i]]:.2f}')

        # Memastikan tidak ada subplot yang kosong di akhir baris terakhir
        if (i + 1) % samples_per_row == 0:
            plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Panggil fungsi ini setelah Anda melatih model dan ingin menampilkan hasil validasi
display_validation_results(X_val, y_bbox_val, y_class_val, classes_list, model, num_samples=10, samples_per_row=5)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Prediksi kelas pada data validasi
y_pred = model.predict(X_val)

# Mendapatkan kelas prediksi dengan nilai tertinggi
y_pred_class = np.argmax(y_pred[1], axis=1)

# Menampilkan confusion matrix
cm = confusion_matrix(y_class_val, y_pred_class)
print("Confusion Matrix:")
print(cm)

# Menampilkan classification report
cr = classification_report(y_class_val, y_pred_class, target_names=classes_list)
print("Classification Report:")
print(cr)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk memuat dan menampilkan gambar dengan prediksi bounding box
def validate_single_image(image_path, model, classes_list):
    # Memuat gambar dari path
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konversi ke format RGB

    # Reshape gambar untuk model
    input_image = cv2.resize(image, (320, 320))  # Ubah sesuai dengan dimensi masukan model Anda
    input_image = np.expand_dims(input_image, axis=0)

    # Melakukan prediksi menggunakan model
    predictions = model.predict(input_image)
    bbox_predictions, class_predictions = predictions[0], predictions[1]

    # Mengambil hasil prediksi untuk gambar pertama
    bbox_pred = bbox_predictions[0]
    class_pred = np.argmax(class_predictions[0])

    # Menampilkan gambar dengan bounding box dan label prediksi
    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)

    # Menggambar bounding box berdasarkan prediksi
    xmin, ymin, xmax, ymax = bbox_pred
    plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                      edgecolor='green', facecolor='none', linewidth=2))
    plt.title(f'Predicted Class: {classes_list[class_pred]}')
    plt.axis('off')
    plt.show()

# Contoh penggunaan
# Pastikan path gambar sesuai dengan lokasi gambar yang ingin Anda uji
image_path = '/content/drive/MyDrive/Dataset/arya5/images/IMG-20240622-WA0013_jpg.rf.6cb01f7f6d0b91d12075f864a134ae25.jpg'  # Ganti dengan path gambar Anda
validate_single_image(image_path, model, classes_list)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import json

# Muat model yang telah dilatih
model_path = '/content/drive/MyDrive/Dataset/arya6/model_checkpoint2.h5'
model = load_model(model_path)

# Muat daftar kelas dari file JSON
with open('/content/drive/MyDrive/Dataset/arya6/classes_list.json', 'r') as f:
    classes_list = json.load(f)

# Fungsi untuk memuat gambar uji
def load_test_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (320, 320))
    return image

# Fungsi untuk pra-pemrosesan gambar
def preprocess_image(image):
    #image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Fungsi untuk membuat prediksi
def make_predictions(model, image):
    bbox_pred, class_pred = model.predict(image)
    class_pred = np.argmax(class_pred, axis=-1)
    return bbox_pred, class_pred

# Fungsi untuk menampilkan hasil
def display_test_image(image_path, bbox_pred, class_pred, classes_list):
    # Debugging: Periksa prediksi kelas dan daftar kelas
    print(f"class_pred: {class_pred}")
    print(f"classes_list: {classes_list}")

    if class_pred[0] < len(classes_list):
        label = classes_list[class_pred[0]]
    else:
        label = "Unknown"

    image = cv2.imread(image_path)
    image = cv2.resize(image, (320, 320))
    bbox = bbox_pred[0]

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                                      edgecolor='red', facecolor='none', linewidth=2))
    plt.title(label)
    plt.show()

# Path ke gambar uji
test_image_path = '/content/drive/MyDrive/Dataset/validasi/WhatsApp Image 2024-07-13 at 01.14.28_420f6806.jpg'

# Muat dan pra-pemroses gambar uji
test_image = load_test_image(test_image_path)
preprocessed_image = preprocess_image(test_image)

# Buat prediksi
bbox_pred, class_pred = make_predictions(model, preprocessed_image)

# Tampilkan hasil
display_test_image(test_image_path, bbox_pred, class_pred, classes_list)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import json
from google.colab.patches import cv2_imshow
from IPython.display import display, Image
import threading
import ipywidgets as widgets

# Load the trained model
model_path = '/content/drive/MyDrive/Dataset/arya6/model_checkpoint2.h5'
model = load_model(model_path)

# Load class list from JSON file
with open('/content/drive/MyDrive/Dataset/arya6/classes_list.json', 'r') as f:
    classes_list = json.load(f)

# Function to preprocess the frame
def preprocess_frame(frame):
    frame = cv2.resize(frame, (320, 320))
    frame = np.expand_dims(frame, axis=0)
    return frame

# Function to make predictions
def make_predictions(model, frame):
    bbox_pred, class_pred = model.predict(frame)
    class_pred = np.argmax(class_pred, axis=-1)
    return bbox_pred, class_pred

# Function to display the frame with bounding box and label
def display_frame(frame, bbox_pred, class_pred, classes_list):
    if class_pred[0] < len(classes_list):
        label = classes_list[class_pred[0]]
    else:
        label = "Unknown"

    bbox = bbox_pred[0]
    cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
    cv2.putText(frame, label, (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    return frame

# Initialize the webcam
camera = cv2.VideoCapture(0)



    # preprocessed_frame = preprocess_frame(frame)
    # bbox_pred, class_pred = make_predictions(model, preprocessed_frame)
    # frame = display_frame(frame, bbox_pred, class_pred, classes_list)



# cv2_imshow(frame)

stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

cv2.destroyAllWindows()
cv2.waitKey(1)


In [ ]:
cap.release()

In [ ]:
from tensorflow.keras.models import load_model

# Path ke model yang telah dilatih (.h5)
model_path = '/content/drive/MyDrive/Dataset/arya5/model_checkpoint2.h5'
model = load_model(model_path)

In [ ]:
# Mendapatkan jumlah kelas dari model
num_classes = model.layers[-1].output_shape[-1]
print(f"Jumlah kelas dalam model: {num_classes}")


In [ ]:
# Pastikan bahwa output layer adalah layer Dense dengan aktivasi softmax
output_layer = model.layers[-1]

# Ambil bobot (weights) dan bias dari output layer
weights, bias = output_layer.get_weights()

# Jika model menggunakan softmax untuk klasifikasi multikelas
if output_layer.activation == 'softmax':
    # Ambil daftar kelas dari bobot
    classes_list = list(output_layer.get_weights()[1])

# Cetak semua kelas yang ada dalam model
print("Kelas yang ada dalam model:")
for i, class_name in enumerate(classes_list):
    print(f"Kelas {i + 1}: {class_name}")
